In [1]:
import cv2
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
def show_image(image, title="Image"):
    """Display an image with cv2
    
    Args:
        image (ndarray): An imag to show
        title (str): A window title.
    """
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Transform image

In [185]:
def transform(target, matrix):
    """Transform target image with transform matrix
    
    Args:
        target (ndarray): image to transform
        matrix (list, ndarray): Either list of 6 floats or a numpy array of shape 2,3 - transform matrix
    """
    error = ValueError("Error: matrix must be a list of 6 elements or ndarraray of shape (2,3)")
    if type(matrix) is list:
        if len(matrix) != 6:
            raise error
        matrix = np.array(matrix).reshape(2,3)
    elif not (hasattr(matrix, 'shape') and matrix.shape == (2,3)):
        raise error
    return cv2.warpAffine(target, matrix, target.shape[:2][::-1])

# Cut the pice of image

In [121]:
def cut_off(target, roi_corners):
    """Cut off a roi polygon from image
    
    Args:
        target (ndarray): target image
        roi_corners (list): list of polygon lists
        
    Returns:
        A numpy array with roi cropped to its bounding box and mask
    """
    
    mask = np.zeros(target.shape, dtype=np.uint8)
    channel_count = target.shape[2]
    ignore_mask_color = (255,)*channel_count
    roi_corners = np.array([roi_corners], dtype=np.int32)
    
    _ = cv2.fillPoly(mask, roi_corners, ignore_mask_color)
    masked_image = cv2.bitwise_and(target, mask)
    x,y,w,h = cv2.boundingRect(mask[:,:,1])
    return masked_image[x:x+w,y:y+h,:], mask[x:x+w, y:y+h,1]

# Put the pice on top

In [148]:
def paste_pice(target, pice, mask, pos):
    """Paste pice on top of the target picture with mask.
    
    Args:
        target (ndarray): target picture
        pice (ndarray): picture to paste
        mask (ndarray): picture mask
        pos (tuple): (x, y) position on target picture to put pice on
    """
    x,y = pos
    w,h = pice.shape[:2]
    mask_inv = cv2.bitwise_not(mask)
    roi = target[x:x+w, y:y+h]
    blacked = cv2.bitwise_and(roi, roi, mask = mask_inv)
    dst = cv2.add(blacked,pice)
    target[x:x+w, y:y+h] = dst

In [191]:
target = cv2.imread('2.jpg', cv2.IMREAD_UNCHANGED)

In [192]:
show_image(target)

In [193]:
pice, mask = cut_off(target, [(100,100), (500,200), (400,400), (200, 500)])

In [194]:
show_image(pice)

In [195]:
t = target.copy()

In [196]:
paste_pice(t, pice, mask, (50,250))

In [197]:
show_image(t)